In [102]:
import pandas as pd
import numpy as np

In [103]:
fe = pd.read_csv('Fashione-EAN-PLU.csv', dtype=str)
gs1 = pd.read_csv("01 All barcodes.csv", skiprows=3, dtype=str)
vs = pd.read_csv('data-for-gs1-from-vs.csv', dtype=str)

fe_temp = pd.read_csv('Fashione-EAN-PLU.csv', dtype=str)
gs1_temp = pd.read_csv("01 All barcodes.csv", skiprows=3, dtype=str)
vs_temp = pd.read_csv('data-for-gs1-from-vs.csv', dtype=str)

In [104]:
# vs_temp['Product Name'] = vs_temp['Parent Product Title']
# vs_temp['SKU'] = vs_temp['Child Reference']
# vs_temp = vs_temp[['Product Name', 'SKU', 'Categories']]
# gs1_temp = pd.merge(gs1_temp, vs_temp, on='Product Name', how='left')
# gs1_temp['SKU_y'] = gs1_temp['SKU_y'].fillna(gs1_temp['SKU_x'])
# gs1_temp.rename(columns={'SKU_y': 'PLU'}, inplace=True)
# gs1_temp = gs1_temp[['PLU', 'Number']]

In [105]:
# print(fe['EAN'].isnull().sum())
# fe = pd.merge(fe, gs1_temp, on='PLU', how='left')
# fe['EAN_temp'] = fe['EAN'].fillna(fe['Number'])
# print(fe['EAN_temp'].isnull().sum())
# list(set(fe['EAN_temp'].values) - set(fe['EAN'].values))

In [106]:
# fe[fe['Number'].isin(list(set(fe['EAN_temp'].values) - set(fe['EAN'].values)))].drop(['Number'], axis=1).to_csv('Temp_Fashione_EAN_Fix.csv', index=False)

In [107]:
vs['Parent Product Url'] = vs['Parent Product Url'].str.replace('https://www.leathercompany.co.uk', '')
vs['Parent Product Url'] = 'https://www.leathercompany.co.uk' + vs['Parent Product Url']

vs.rename(columns={'Child Reference': 'PLU'}, inplace=True)
vs = vs[['PLU', 'Parent Product Url', 'Child Product Title']]

fe = pd.merge(fe, vs, on='PLU', how='left')

In [108]:
print(gs1.shape)
gs1.drop_duplicates(keep=False, inplace=True)
print(gs1.shape)

(6005, 10)
(6000, 10)


In [109]:
fe = fe.dropna(subset=['EAN'])
fe['Size'] = fe['Size'].replace(np.NaN, 'ONE SIZE')
fe['Style-Colour-Size'] = fe['Style'] + ' | ' + fe['Colour'] + ' | Size:' + fe['Size']
fe['Child Product Title'] = fe['Child Product Title'].replace(np.NaN, '')
fe['Temp_Description'] = fe['Style-Colour-Size'] + ' | ' + fe['Child Product Title']
fe = fe[['PLU', 'EAN', 'Style-Colour-Size', 'Parent Product Url', 'Temp_Description']]
gs1.rename(columns={'Number': 'EAN'}, inplace=True)

In [110]:
df = pd.merge(gs1, fe, on='EAN', how='left')

nosku_df = df[df['PLU'].isna()]
df = df[~df['PLU'].isna()]

temp = df.dropna(subset=['Style-Colour-Size'])
dict_name_ean = pd.Series(temp['Style-Colour-Size'].values,index=temp['EAN']).to_dict()
dict_wrongname_ean = pd.Series(df['Product Name'].values,index=df['EAN']).to_dict()
dict_name_ean = dict_wrongname_ean | dict_name_ean
df['Product Name'] = df['EAN']
df['Product Name'] = df['Product Name'].map(dict_name_ean)
temp = df.dropna(subset=['Temp_Description'])
dict_descr_ean = pd.Series(temp['Temp_Description'].values,index=temp['EAN']).to_dict()
dict_wrongdescr_ean = pd.Series(df['Description'].values,index=df['EAN']).to_dict()
dict_descr_ean = dict_wrongdescr_ean | dict_descr_ean
df['Description'] = df['EAN']
df['Description'] = df['Description'].map(dict_descr_ean)
dict_plu_ean = pd.Series(df['PLU'].values,index=df['EAN']).to_dict()
df['SKU'] = df['EAN']
df['SKU'] = df['SKU'].map(dict_plu_ean)
temp = df.dropna(subset=['Product Link'])
dict_url_ean = pd.Series(temp['Product Link'].values,index=temp['EAN']).to_dict()
dict_wrongurl_ean = pd.Series(df['Product Link'].values,index=df['EAN']).to_dict()
dict_url_ean = dict_wrongurl_ean | dict_url_ean
df['Product Link'] = df['EAN']
df['Product Link'] = df['Product Link'].map(dict_url_ean)

temp = df.dropna(subset=['Product Link'])
temp['Root PLU'] = temp['PLU'].str[:-2]
dict_url_rplu = pd.Series(temp['Parent Product Url'].values,index=temp['Root PLU']).to_dict()
df['Root PLU'] = df['PLU'].str[:-2]
df['Product Link'] = df['Root PLU']
df['Product Link'] = df['Product Link'].map(dict_url_rplu)

df['Main Brand'] = 'Ashwood'
df['Sub Brand'] = 'N/A'
df['MPN'] = 'N/A'
df.drop(['Style-Colour-Size', 'PLU', 'Parent Product Url', 'Temp_Description', 'Root PLU'], axis=1, inplace=True)
nosku_df.drop(['Style-Colour-Size', 'PLU', 'Parent Product Url', 'Temp_Description'], axis=1, inplace=True)

df = pd.concat([df, nosku_df], ignore_index=True)

df.shape

C:\Users\retai\AppData\Local\Temp\ipykernel_21820\2399808823.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Root PLU'] = temp['PLU'].str[:-2]


(6000, 10)

In [111]:
df = pd.merge(gs1, fe, on='EAN', how='left')

nosku_df = df[df['PLU'].isna()]
df = df[~df['PLU'].isna()]

temp = df.dropna(subset=['Style-Colour-Size'])
dict_name_ean = pd.Series(temp['Style-Colour-Size'].values,index=temp['EAN']).to_dict()
dict_wrongname_ean = pd.Series(df['Product Name'].values,index=df['EAN']).to_dict()
dict_name_ean = dict_wrongname_ean | dict_name_ean
df['Product Name'] = df['EAN']
df['Product Name'] = df['Product Name'].map(dict_name_ean)
temp = df.dropna(subset=['Temp_Description'])
dict_descr_ean = pd.Series(temp['Temp_Description'].values,index=temp['EAN']).to_dict()
dict_wrongdescr_ean = pd.Series(df['Description'].values,index=df['EAN']).to_dict()
dict_descr_ean = dict_wrongdescr_ean | dict_descr_ean
df['Description'] = df['EAN']
df['Description'] = df['Description'].map(dict_descr_ean)
dict_plu_ean = pd.Series(df['PLU'].values,index=df['EAN']).to_dict()
df['SKU'] = df['EAN']
df['SKU'] = df['SKU'].map(dict_plu_ean)
dict_url_ean = pd.Series(df['Parent Product Url'].values,index=df['EAN']).to_dict()
df['Product Link'] = df['EAN']
df['Product Link'] = df['Product Link'].map(dict_url_ean)
df['Main Brand'] = 'Ashwood'
df['Sub Brand'] = 'N/A'
df['MPN'] = 'N/A'
df.drop(['Style-Colour-Size', 'PLU', 'Parent Product Url', 'Temp_Description'], axis=1, inplace=True)
nosku_df.drop(['Style-Colour-Size', 'PLU', 'Parent Product Url', 'Temp_Description'], axis=1, inplace=True)

stock_df = pd.read_csv('stock_levels_02_may.csv', dtype=str)
stock_df = stock_df[['reference', 'stock']]
stock_df.rename(columns={'reference': 'SKU', 'stock': 'Stock'}, inplace=True)
null_url = df[df['Product Link'].isna()]
null_url = pd.merge(null_url, stock_df, on='SKU', how='left')
null_url = null_url[['EAN', 'Product Name', 'SKU', 'Stock']].sort_values(by='Stock', ascending=True)
null_url.to_csv('Null URLs - Unable to match Fashione PLU.csv', index=False)

df['Product Link'] = df['Product Link'].fillna('N/A')

# df = pd.concat([df, nosku_df], ignore_index=True)
df.rename(columns={'EAN': 'Number'}, inplace=True)
df.shape

(2131, 10)

In [112]:
print(null_url.shape)
print(null_url.dropna(subset=['Stock']).shape)

(115, 4)
(0, 4)


In [113]:
df = df.sort_values(by='SKU')

In [114]:
df.isnull().sum()

Number          0
Status          0
Product Name    0
Description     0
Main Brand      0
Sub Brand       0
Product Link    0
MPN             0
SKU             0
Updated         0
dtype: int64

In [115]:
df.to_csv('Fixed SKU-EAN.csv', index=False)

In [116]:
df.shape

(2131, 10)

In [117]:
df_506104960 = df[df["Number"].str.contains("506104960")]
df_506104299 = df[df["Number"].str.contains("506104299")]
df_506102791 = df[df["Number"].str.contains("506102791")]
df_506099970 = df[df["Number"].str.contains("506099970")]
df_506082128 = df[df["Number"].str.contains("506082128")]
df_506047452 = df[df["Number"].str.contains("506047452")]

df_506104960.shape, df_506104299.shape, df_506102791.shape, df_506099970.shape, df_506082128.shape, df_506047452.shape

((671, 10), (618, 10), (227, 10), (460, 10), (101, 10), (54, 10))

In [118]:
671 + 618 + 227 + 460 + 101 + 54

2131

In [119]:
df_506104960.to_csv('Update 506104960.csv', index=False)
df_506104299.to_csv('Update 506104299.csv', index=False)
df_506102791.to_csv('Update 506102791.csv', index=False)
df_506099970.to_csv('Update 506099970.csv', index=False)
df_506082128.to_csv('Update 506082128.csv', index=False)
df_506047452.to_csv('Update 506047452.csv', index=False)